In [ ]:
import pyspark.sql.functions as f

# PySpark DataFrames: Data Analysis Part 1

Welcome to the second part of the PySpark DataFrames module! In this part, we will focus on more advanced operations on DataFrames to perform data analysis and manipulation.

There are plenty of operations that can be performed on DataFrames. Therefore, this notebook will cover the most common and useful ones by answering some questions about the data we've preprocessed in the first part.

This notebook is composed by a set of questions that will teach you how to use some pyspark DataFrame methods and SQL functions that are very useful for data analysis.

To start off, let's get the orders and products data we've saved in the DBFS in the first part of this module.

In [ ]:
df_orders = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ",")
    .load("/FileStore/lp-big-data/orders-data/orders-preprocessed.csv")
)

df_products = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ",")
    .load("/FileStore/lp-big-data/orders-data/products-preprocessed.csv")
)

df_orders.display()
df_products.display()

## Join tables

Our two tables complement each other. The orders table provides information about the orders, each comprising one or more units of a product.

The common column between them is `product_id`. We can merge them based on this column to extract relevant statistics.

But how do we merge two tables?

Spark DataFrames offer similar join operations to those found in SQL.

To use the `.join` operator, we need to specify:
- Two tables, referred to as the left and right tables, respectively.
- One or more predicates, which are the conditions determining how records from the two tables are matched.
- Finally, a method indicating how the join is performed when the predicate succeeds and when it fails.

Here's how it works:

```
left_table.join(
  right_table,
  on=[predicates],
  how='join_method'
)
```

The predicates in a PySpark join are rules applied between columns of the left and right data frames. The join occurs record-wise, comparing each record in the left data frame to each record in the right data frame based on the predicates. If the predicates return `True`, it's a match; otherwise, it's a no-match.


Similarly to SQL, there are several join methods available in PySpark, with the default one being the `inner` join.

The following image summarizes the most common join strategies. For a detailed tutorial on PySpark joins check this [link](https://www.geeksforgeeks.org/pyspark-join-types-join-two-dataframes/).

![Join types](https://github.com/inesmcm26/lp-big-data-mercedes/blob/main/data/images/joins-in-mysql.png?raw=true)

Let's use the join operator to join the orders and products tables.

We want to use a left join on the orders table because we don't want to miss any orders data, even the products sold on that order are not listed on the products table.

In [ ]:
df_orders_products = (
    df_orders.join(
        df_products,
        on=['product_id'],
        how='left'
    )
)

df_orders_products.display()

We joined the two tables using a column that is present in both tables. Because of that, we ended up with a table that has only one column `product_id`, resulting from collapsing these columns from both tables.

But there could be the case that the column we're joining on has not the exact same name in both tables. In that case we would have to specify the name of the column of each table.

We can still do it that way, even if the column we're joining on has the same name in both tables.

In [ ]:
df_orders_products_duplicate = (
    df_orders.join(
        df_products,
        on=df_orders['product_id']==df_products['product_id'],
        how='left'
    )
)

# Or

df_orders_products_duplicate = (
    df_orders.alias('orders')
    .join(
        df_products.alias('products'),
        on=[f.col('orders.product_id') == f.col('products.product_id')],
        how='left'
    )
)

df_orders_products_duplicate.display()

With this method we end up with two `product_id` columns. In this case, we can simply drop one of them. We need to specify which of the columns we want to drop, otherwise, both columns will be dropped.

In [ ]:
df_orders_products = (
    df_orders_products_duplicate
    .drop(df_products['product_id'])
)

df_orders_products.display()

In [ ]:
df_orders_products.printSchema()

## Data Analytics

Now that we have our tables merged, let's answer some questions!

1. How many unique customers placed orders?

In [ ]:
df_orders_products.select("customer_id").distinct().count()

Another way of doing it is by using the `countDistinct()` function.

In [ ]:
df_orders_products.select(f.countDistinct('order_id')).display()

2. What is the total profit generated from all orders?

In [ ]:
# We can do it using an aggregation function
(
    df_orders_products
    .select(
        f.sum(f.col('profit')).alias('total_profit')
    )
).display()

In [ ]:
# Or using RDD functions
profit_list = (
    df_orders_products
    .select(f.col('profit'))
    .rdd.flatMap(lambda x: x)
    .collect()
)

sum(profit_list)

3. When was the first time each customer placed an order?

To answer this question with need to group information by customer and then aggregate the data to get the first order date.

But first, we need to sort the orders by the placing date.

In [ ]:
(
  df_orders_products
  .orderBy('placing_date')
  .groupBy('customer_id')
  .agg(
      f.first('placing_date').alias('first_placing_date')
)
).display()

4. What is the name of the slowest product to be delivered on average?

Remember that we created a column `days_to_delivery`? Let's use it to answer this question.

Once again, we want to group information by product and then apply an aggregation function that gives us the average delivery time.

After that, we can sort the results in descending order to get the slowest product.

In [ ]:
(
    df_orders_products
    .groupBy(['product_id', 'product_name'])
    .agg(
        f.avg('days_to_delivery').alias('avg_delivery_delay')
    )
    .sort(f.desc('avg_delivery_delay'))
    .select('product_name')
    .limit(1)
).display()

5. How many orders with category `Shoes` were placed during the weekends (week days 6 and 7)? 

To answer this question we do not want to aggregate data. We want to filter the data based on the category and the day of the week and then count the number of orders.

In [ ]:
(
    df_orders_products
    .filter(f.col('product_category') == 'Shoes')
    .filter(f.col('order_day_of_week').isin([6, 7]))
).count()

6. What is the ID of the most ordered product for each delivery speed?

Now we need to group the data based on not only one, but two columns.

First, we need to get the number of times each product is ordered for each delivery speed.

Then, we can get the product with the highest number of orders for each delivery speed and the corresponding number of orders.

In [ ]:
# Calculate the count of each product for each delivery speed
product_count_df = (
    df_orders_products
    .groupBy(['delivery_speed', 'product_id'])
    .agg(f.count('order_id').alias('nr_orders'))
)

# Find the product with the maximum number of orders for each delivery speed
result_df = (
    product_count_df
    .orderBy(['delivery_speed', f.desc('nr_orders')])
    .groupBy('delivery_speed')
    .agg(
        f.first('product_id').alias('most_ordered_product_id'),
        f.max('nr_orders').alias('max_orders')
    )
)

result_df.display()

7. Which product category has the highest total sales revenue?

In [ ]:
(
    df_orders_products
    .groupBy('product_category')
    .agg(f.sum('revenue').alias('total_revenue'))
    .sort(f.desc('total_revenue'))
).show(1)

8. For each customer segment, what is the maximum delivery delay and the average profit?

We can perform multiple aggregations on the same group of data at once as we've seen in question 6. Let's see another syntax to do that.

In [ ]:
(
    df_orders_products
    .groupBy('customer_status')
    .agg({
        'days_to_delivery' : 'max',
        'profit' : 'avg',
    })
).display()

9. What is the average number of units (amount) per order of each product sold?

In [ ]:
# Or using
(
    df_orders_products
    .groupBy(f.col('product_id'))
    .agg(f.avg('amount').alias('avg_amount'))
).display()

10. What are the dates in which each product was delivered among all orders?

To answer this question we need to group the data by product and then aggregate the dates in which the product was delivered on a list. We can use the `collect_list()` function to do that.

This aggregation function is different from the ones we've seen so far. Instead of collapsing all the data into a single value, it creates a list of values.

In [ ]:
(
    df_orders_products
    .groupBy(f.col('product_id'))
    .agg(f.collect_list('delivery_date').alias('delivery_dates'))
).display()

11. How many orders of each category were placed in each year?

To answer this question we can use the pivot operation. This operation produces a pivot table which is a cross-tabulation that can show the relationship between two columns.

As an alternative we could also use the groupBy operation, but the pivot operation results in a more readable table.

In [ ]:
# Using groupBy
(
    df_orders_products
    .groupBy('order_year', 'product_category')
    .agg(f.count('order_id'))
).display()

In [ ]:
# Using pivot
pivot_df = (
    df_orders_products
    .groupBy('order_year')
    .pivot('product_category')
    .agg(f.count('order_id'))
)

pivot_df.display()

This query created one row each element in the `groupBy` clause and one column for each unique value in the `pivot` clause.

12. What is the total profit each continent suppliers generated each year?

Again, we can use the pivot operation to answer this question.

In [ ]:
pivot_df = (
    df_orders_products
    .groupBy('order_year')
    .pivot('supplier_continent')
    .agg(f.sum('profit'))
)

pivot_df.display()

13. Who is the client that bought the greatest variety of products in each year?

Let's break the question down into smaller parts:
- First, we need to count the number of unique products each customer bought in each year.
- Then, we need to get the customer with the highest number of unique products bought in each year.

In [ ]:
# Find the number of unique products each client ordered in each year
unique_product_count_df = (
    df_orders_products
    .groupBy(['customer_id', 'order_year'])
    .agg(f.countDistinct('product_id').alias('nr_unique_products'))
)

# Find the customer with the maximum number of unique products for each year
result_df = (
    unique_product_count_df
    .orderBy(['order_year', f.desc('nr_unique_products')])
    .groupBy('order_year')
    .agg(f.first('customer_id').alias('most_varied_customer'), f.max('nr_unique_products').alias('max_unique_products'))
)

result_df.display()

14. What is the standard deviation of profit for each supplier?

In [ ]:
(
    df_orders_products
    .groupBy('supplier_id')
    .agg(f.stddev('profit').alias('profit_stddev'))
).display()

15. How many unique customers placed orders within each supplier continent?

In [ ]:
(
    df_orders_products
    .groupBy('supplier_continent')
    .agg(
        f.countDistinct('customer_id')
    )
).display()

---

Great! We've gone through a lot of questions and have used several of the most used PySpark DataFrame methods and SQL functions.

Now it's time to solve some exercises to practice what we've learned.

Go to notebook `exercises-part2` to solve the exercises.